In [1]:
import numpy as np
import pandas as pd
import os
from openfisca_survey_manager.survey_collections import SurveyCollection
from openfisca_france_indirect_taxation.utils import assets_directory

In [2]:
from configparser import ConfigParser
import datetime
import os
import platform
import pandas
import subprocess

from openfisca_france_indirect_taxation.examples.utils_example import df_weighted_average_grouped, collapse
    
from openfisca_france_indirect_taxation.utils import assets_directory, get_input_data_frame
from openfisca_france_indirect_taxation.build_survey_data.matching_bdf_entd.step_4_1_save_data import prepare_bdf_entd_matching_data
from openfisca_france_indirect_taxation.build_survey_data.matching_bdf_enl.step_4_1_clean_data import prepare_bdf_enl_matching_data
from openfisca_france_indirect_taxation.build_survey_data.matching_bdf_entd.step_6_1_calage_depenses_carburants import cale_bdf_entd_matching_data
from openfisca_france_indirect_taxation.build_survey_data.step_5_data_from_matching import check_load_config_ini

In [4]:
from openfisca_france_indirect_taxation.build_survey_data.matching_bdf_enl.step_2_homogenize_variables import create_new_variables, homogenize_variables_definition_bdf_enl


In [4]:
path_to_r_libs_user, path_to_rscript_exe = check_load_config_ini()
prepare_bdf_enl_matching_data(2017)
r_script_path = os.path.join(assets_directory, 'matching', 'matching_enl', 'matching_rank_bdf_enl.R')
if path_to_r_libs_user is not None:
    os.environ['R_LIBS_USER'] = path_to_r_libs_user
process_call = [path_to_rscript_exe, '--vanilla', r_script_path]
subprocess.call(process_call)

c:\users\veve1\documents\openfisca-france-indirect-taxation\openfisca_france_indirect_taxation\Calage_consommation_bdf.py:171: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_bdf_postes_cn[poste] = 0


0

In [55]:
data_matched = pd.read_csv(
    os.path.join(
        assets_directory,
        'matching',
        'matching_entd',
        'data_matched_distance.csv'
        ), sep =',', decimal = '.'
    )

grouped = data_matched.groupby(['niveau_vie_decile', 'rural'])

results = []
for (decile, rur), group in grouped:
    weight_sum = group['pondmen'].sum()
    
    # weighted averages
    avg_distance = (group['distance'] * group['pondmen']).sum() / weight_sum
    avg_depenses = (group['poste_07_2_2_1'] * group['pondmen']).sum() / weight_sum
    
    # correction factor
    factor = avg_depenses / avg_distance if avg_distance != 0 else 0
    
    results.append({
        'niveau_vie_decile': decile,
        'rural': rur,
        'avg_distance': avg_distance,
        'avg_depenses': avg_depenses,
        'factor': factor
    })

results_df = pd.DataFrame(results)


In [56]:
results_df

,niveau_vie_decile,rural,avg_distance,avg_depenses,factor
0,1,0,6624.679468,552.173681,0.083351
1,1,1,11845.497590,952.584245,0.080417
2,2,0,7685.130937,660.408633,0.085933
3,2,1,14800.973325,1297.203401,0.087643
4,3,0,9931.586410,679.788078,0.068447
5,3,1,16523.906088,1406.826398,0.085139
6,4,0,10685.321638,855.590525,0.080072
7,4,1,16373.535085,1518.124005,0.092718
8,5,0,11803.949968,1073.316573,0.090929
9,5,1,19093.745573,1405.657867,0.073619


In [57]:
grouped = input_bdf.groupby(['niveau_vie_decile', 'rural'])

results = []
for (decile, rur), group in grouped:
    weight_sum = group['pondmen'].sum()
    
    # weighted averages
    avg_distance = (group['distance'] * group['pondmen']).sum() / weight_sum
    avg_depenses = (group['poste_07_2_2_1'] * group['pondmen']).sum() / weight_sum
    
    # correction factor
    factor = avg_depenses / avg_distance if avg_distance != 0 else 0
    
    results.append({
        'niveau_vie_decile': decile,
        'rural': rur,
        'avg_distance': avg_distance,
        'avg_depenses': avg_depenses,
        'factor': factor
    })

results_df = pd.DataFrame(results)

In [58]:
results_df

,niveau_vie_decile,rural,avg_distance,avg_depenses,factor
0,1.0,0.0,7338.009036,555.367209,0.075684
1,1.0,1.0,11309.787983,963.680662,0.085208
2,2.0,0.0,8301.846213,663.415293,0.079912
3,2.0,1.0,14178.824788,1295.069062,0.091338
4,3.0,0.0,9320.164273,679.788078,0.072937
5,3.0,1.0,14513.195968,1406.826398,0.096934
6,4.0,0.0,10685.817963,855.590525,0.080068
7,4.0,1.0,13819.559948,1518.124005,0.109853
8,5.0,0.0,11229.989178,1073.316573,0.095576
9,5.0,1.0,16709.925213,1405.657867,0.084121


In [5]:
data_matched_entd = pandas.read_csv(
    os.path.join(
        assets_directory,
        'matching',
        'matching_entd',
        'data_matched_final.csv'
        ), sep =',', decimal = '.'
    )

In [6]:
data_matched_entd.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'X', 'aba', 'age_vehicule', 'agepr',
       'cataeu', 'cs42pr', 'dip14pr', 'etat_veh_achat', 'ident_men',
       'km_autre_carbu', 'km_diesel', 'km_essence', 'mloy_d', 'nactifs',
       'nbphab', 'nenfants', 'npers', 'ocde10', 'pondmen', 'poste_07_2_2_1',
       'prix_achat', 'rev_disponible', 'revtot', 'situapr', 'stalog', 'tau',
       'tuu', 'typmen', 'veh_tot', 'vp_deplacements_pro',
       'vp_domicile_travail', 'rural', 'petite_ville', 'moyenne_ville',
       'grande_ville', 'paris', 'nb_essence', 'nb_diesel', 'niveau_vie',
       'niveau_vie_decile', 'niveau_vie_quintile', 'donation_class_3',
       'distance', 'distance_autre_carbu', 'distance_diesel',
       'distance_essence', 'age_vehicule.1',
       'depenses_carburants_corrigees_entd', 'depenses_diesel_corrigees_entd',
       'depenses_essence_corrigees_entd'],
      dtype='object')